# 事件驱动计算

事件驱动是类脑仿真计算模型有别于其他传统计算模型的重要特征，大脑的神经元只有在接收脉冲事件后才会被激活并计算，每个时刻需要计算的神经元集合可以被看做一个稀疏向量，这意味着传统矩阵乘法的方式已不适用于计算神经元的发放过程。

![](../_static/dense-mv-vs-event-spmv.png)

BrainState对事件驱动这一特性提供了专门的算子优化，使得模型在处理稀疏数据的情况下，能够大幅度减少计算资源的使用并提高速度。同时BrainState为神经元之间的突触连接提供几种连接方式，以下是相关的示例。

In [5]:
import jax
import jax.numpy as jnp

import brainstate

### 1.固定概率连接

有些时候我们并不知道神经元之间的具体连接，但我们知道对于每个突触前神经元，它连接突触后神经元群体有一个固定的概率。

`BrainState`提供类`brainstate.event.FixedProb`来定义这种连接过程。

类`brainstate.event.FixedProb`接受以下参数：
- `n_pre`: 突触前神经元的数量
- `n_post`: 突触后神经元的数量
- `prob`: 每个突触前神经元与突触后神经元连接的固定概率
- `weight`: 最大突触电导，接受单个浮点数、数组或者函数，支持标量和物理量
- `allow_multi_conn`: 突触前神经元能否与同一个突触后神经元有多个链接，默认值`True`
- `seed`:  随机生成种子，默认值`None`
- `name`: 模块名称，默认值`None`
- `grad_mode`: 自动微分方法，默认值`vjp`


In [6]:
# pre-synaptic spikes, 10000 neurons, 1% sparsity
pre_spikes = brainstate.random.rand(10000) < 0.01

In [7]:
# dense weight matrix, 10000x1000, 1% sparsity
dense_w = (brainstate.random.rand(10000, 1000) < 0.01).astype(float)

# event-driven weight matrix, 10000x1000, 1% sparsity
fp = jax.jit(brainstate.event.FixedProb(10000, 1000, 0.01, 1.))

In [8]:
# 先编译运行一次，随后测平均运行时间
jnp.dot(pre_spikes, dense_w)
%timeit -n 100 -r 10 jnp.dot(pre_spikes, dense_w)

fp(pre_spikes)
%timeit -n 100 -r 10 fp(pre_spikes)

1.28 ms ± 228 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
409 μs ± 4.73 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


### 2.全连接

神经元之间的突触连接是一个密集矩阵。

`BrainState`提供类`brainstate.event.Linear`来定义这种连接过程。

类`brainstate.event.Linear`接受以下参数：
- `n_pre`: 突触前神经元的数量
- `n_post`: 突触后神经元的数量
- `weight`: 最大突触电导，接受单个浮点数、数组或者函数，支持标量和物理量
- `name`: 模块名称，默认值`None`
- `grad_mode`: 自动微分方法，默认值`vjp`

In [9]:
# dense weight matrix, 10000x1000 
dense_w = brainstate.random.rand(10000, 1000).astype(float)

# event-driven weight matrix, 10000x1000
fp = jax.jit(brainstate.event.Linear(10000, 1000, dense_w))

In [10]:
# 先编译运行一次，随后测平均运行时间
jnp.dot(pre_spikes, dense_w)
%timeit -n 100 -r 10 jnp.dot(pre_spikes, dense_w)

fp(pre_spikes)
%timeit -n 100 -r 10 fp(pre_spikes)

1.33 ms ± 335 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
390 μs ± 38.5 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


### 3.稀疏矩阵连接

大部分情况下，神经元之间的突触连接是一个稀疏矩阵，`CSR`格式是存储稀疏矩阵的一种常见格式，其表示形式如下图：

![](../_static/csr_matrix.png)

`CSR`格式由三个Numpy数组组成：`indices`, `indptr`, `values`:
- `indices`: 记录矩阵每个非零元素的列坐标，先按行排序，再按列排序
- `indptr`: 长度为`row+1`, `indptr[i]`表示第i行非零元素在`indices`上的起始位置
- `values`: 记录`indices`数组对应非零元素的值


BrainState提供类`brainstate.event.CSRLinear`来定义这种连接过程。

类`brainstate.event.CSRLinear`接受以下参数：
- `n_pre`: 突触前神经元的数量
- `n_post`: 突触后神经元的数量
- `indptr`: 稀疏矩阵CSR格式的`indptr`数组
- `indices`: 稀疏矩阵CSR格式的`indices`数组
- `weight`: 最大突触电导，接受单个浮点数、数组或者函数，支持标量和物理量
- `name`: 模块名称，默认值`None`
- `grad_mode`: 自动微分方法，默认值`vjp`

In [11]:
# dense weight matrix, 10000x10000, 0.01% sparsity
dense_w = (brainstate.random.rand(10000, 10000) < 0.0001).astype(float)

# event-driven weight matrix, 10000x1000, 0.01% sparsity, CSR format
from scipy.sparse import csr_matrix

csr = csr_matrix(dense_w)
fp = jax.jit(brainstate.event.CSRLinear(10000, 10000, csr.indptr, csr.indices, csr.data))

In [12]:
# 先编译运行一次，随后测平均运行时间
jnp.dot(pre_spikes, dense_w)
%timeit -n 100 -r 10 jnp.dot(pre_spikes, dense_w)

fp(pre_spikes)
%timeit -n 100 -r 10 fp(pre_spikes)

13.5 ms ± 975 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
3.18 ms ± 922 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


### 4.突触权重

在建模突触连接时，`weight`参数接受单个浮点数、数组或者函数，支持标量和物理量。

如果传入`weight`的是一个浮点数，说明所有突触权重相同，存储单个浮点数可以极大程度节省内存空间。

如果传入`weight`的是一个数组，则要保证数组能够与连接矩阵对齐，即数组与连接矩阵大小相同，或者连接矩阵能被完美切分成多个数组。

如果传入`weight`的是一个函数，则会在`init`阶段生成连接矩阵大小的权重矩阵。

引用`BrainUnit`库后，`BrainState`能够支持物理量输入和计算。